In [1]:
import pandas as pd
import dask.dataframe as dd

from category_encoders.binary import BinaryEncoder

## Функция для чтения данных

In [2]:
def read_data(path,columns):
    data = pd.DataFrame()
    for name_column, column_type in columns.items():
        df_column = pd.DataFrame()
        for i in range(10):
            data_features_file = dd.read_parquet(
                path + f'\\part-0000{i}-aba60f69-2b63-4cc1-95ca-542598094698-c000.snappy.parquet')
            df_column = pd.concat([df_column, data_features_file[name_column].compute()], ignore_index=True)
        data[name_column] = df_column.astype(column_type)
    return data

## Популярный регион, всего регионов binary encoder

In [6]:
path = '..\\full_data\\competition_data_final_pqt'
columns = {'user_id': 'int32', 'region_name': 'category'}
data = read_data(path,columns)
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 322899435 entries, 0 to 322899434
Data columns (total 2 columns):
 #   Column       Dtype   
---  ------       -----   
 0   user_id      int32   
 1   region_name  category
dtypes: category(1), int32(1)
memory usage: 1.5 GB


In [34]:
# в top_region количество строк с этим регионом у юзера
df = data.groupby(['user_id', 'region_name'])['region_name'].count()
df.name = 'top_region'

display(df.info())
print(df.index)

# удаляем из фрейма города, в которых не был человек
df = df[df != 0].reset_index()
display(df.head(20))
# в скольки регионах бывал пользователь
region_count = df.groupby('user_id', group_keys=False)['region_name'].count()
#оставляем только топовые регионы для каждого пользоватеоля
df = df.sort_values(by='top_region', ascending=False).drop_duplicates(subset='user_id', keep='first', ignore_index=True)
df = df.drop('top_region', axis=1)
#сортируем и добавляем колонку
df = df.sort_values(by='user_id').reset_index(drop=True)
df['region_count'] = region_count

<class 'pandas.core.series.Series'>
MultiIndex: 33640677 entries, (0, 'Алтайский край') to (415316, 'Ярославская область')
Series name: top_region
Non-Null Count     Dtype
--------------     -----
33640677 non-null  int64
dtypes: int64(1)
memory usage: 436.4 MB


None

MultiIndex([(     0,             'Алтайский край'),
            (     0,           'Амурская область'),
            (     0,      'Архангельская область'),
            (     0,       'Астраханская область'),
            (     0,       'Белгородская область'),
            (     0,           'Брянская область'),
            (     0,       'Владимирская область'),
            (     0,      'Волгоградская область'),
            (     0,        'Вологодская область'),
            (     0,        'Воронежская область'),
            ...
            (415316,      'Удмуртская Республика'),
            (415316,        'Ульяновская область'),
            (415316,           'Хабаровский край'),
            (415316, 'Ханты-Мансийский АО — Югра'),
            (415316,        'Челябинская область'),
            (415316,       'Чеченская Республика'),
            (415316,       'Чувашская Республика'),
            (415316,               'Чукотский АО'),
            (415316,          'Ямало-Ненецкий АО

,user_id,region_name,top_region
0,0,Москва,131
1,1,Москва,572
2,1,Московская область,37
3,1,Санкт-Петербург,91
4,2,Республика Коми,356
5,3,Воронежская область,188
6,4,Волгоградская область,23
7,4,Краснодарский край,459
8,4,Республика Башкортостан,97
9,4,Самарская область,11


In [35]:
display(df)
be = BinaryEncoder(cols=['region_name'],return_df=True)
df = be.fit_transform(df)
display(df)

,user_id,region_name,region_count
0,0,Москва,1
1,1,Москва,3
2,2,Республика Коми,1
3,3,Воронежская область,1
4,4,Краснодарский край,5
...,...,...,...
415312,415312,Тамбовская область,1
415313,415313,Саратовская область,1
415314,415314,Новосибирская область,1
415315,415315,Краснодарский край,1


,user_id,region_name_0,region_name_1,region_name_2,region_name_3,region_name_4,region_name_5,region_name_6,region_count
0,0,0,0,0,0,0,0,1,1
1,1,0,0,0,0,0,0,1,3
2,2,0,0,0,0,0,1,0,1
3,3,0,0,0,0,0,1,1,1
4,4,0,0,0,0,1,0,0,5
...,...,...,...,...,...,...,...,...,...
415312,415312,1,0,0,0,1,1,0,1
415313,415313,0,0,1,1,0,0,0,1
415314,415314,0,0,1,0,0,1,1,1
415315,415315,0,0,0,0,1,0,0,1


In [36]:
display(df.info())
print(df['region_count'].max()) #int8
print(df['user_id'].max()) #int32

df[['region_count']] = df[['region_count']].astype('int8')
df[['user_id']] = df[['user_id']].astype('int32')
df[df.columns[~df.columns.isin(['region_count','user_id'])]] = df[df.columns[~df.columns.isin(['region_count','user_id'])]].astype('category')

display(df.info())


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 415317 entries, 0 to 415316
Data columns (total 9 columns):
 #   Column         Non-Null Count   Dtype
---  ------         --------------   -----
 0   user_id        415317 non-null  int64
 1   region_name_0  415317 non-null  int64
 2   region_name_1  415317 non-null  int64
 3   region_name_2  415317 non-null  int64
 4   region_name_3  415317 non-null  int64
 5   region_name_4  415317 non-null  int64
 6   region_name_5  415317 non-null  int64
 7   region_name_6  415317 non-null  int64
 8   region_count   415317 non-null  int64
dtypes: int64(9)
memory usage: 28.5 MB


None

41
415316
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 415317 entries, 0 to 415316
Data columns (total 9 columns):
 #   Column         Non-Null Count   Dtype   
---  ------         --------------   -----   
 0   user_id        415317 non-null  int32   
 1   region_name_0  415317 non-null  category
 2   region_name_1  415317 non-null  category
 3   region_name_2  415317 non-null  category
 4   region_name_3  415317 non-null  category
 5   region_name_4  415317 non-null  category
 6   region_name_5  415317 non-null  category
 7   region_name_6  415317 non-null  category
 8   region_count   415317 non-null  int8    
dtypes: category(7), int32(1), int8(1)
memory usage: 4.8 MB


None

In [38]:
df.to_parquet('..\\full_data\\figma_plan\\region.parquet', index=False)

## Популярный город, всего городов binary encoder

In [3]:
path = '..\\full_data\\competition_data_final_pqt'
columns = {'user_id': 'int32', 'city_name': 'category'}
data = read_data(path,columns)
display(data.info())
display(data.isna().sum())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 322899435 entries, 0 to 322899434
Data columns (total 2 columns):
 #   Column     Dtype   
---  ------     -----   
 0   user_id    int32   
 1   city_name  category
dtypes: category(1), int32(1)
memory usage: 1.8 GB


In [6]:
df = data.groupby(['user_id', 'city_name'])['city_name'].count()
df.name = 'top_city'

display(df.info())
print(df.index)

df = df[df != 0].reset_index()
display(df.head(20))

city_count = df.groupby('user_id', group_keys=False)['city_name'].count()

df = df.sort_values(by='top_city', ascending=False).drop_duplicates(subset='user_id', keep='first', ignore_index=True)
df = df.drop('top_city', axis=1)

df = df.sort_values(by='user_id').reset_index(drop=True)
df['city_count'] = city_count

<class 'pandas.core.series.Series'>
MultiIndex: 409087245 entries, (0, 'Абаза') to (415316, 'Яхрома')
Series name: top_city
Non-Null Count      Dtype
--------------      -----
409087245 non-null  int64
dtypes: int64(1)
memory usage: 5.4 GB


None

MultiIndex([(     0,      'Абаза'),
            (     0,     'Абакан'),
            (     0,   'Абдулино'),
            (     0,     'Абинск'),
            (     0,    'Агидель'),
            (     0,      'Агрыз'),
            (     0,   'Адыгейск'),
            (     0,  'Азнакаево'),
            (     0,       'Азов'),
            (     0, 'Ак-Довурак'),
            ...
            (415316,     'Якутск'),
            (415316, 'Ялуторовск'),
            (415316,      'Янаул'),
            (415316,     'Яранск'),
            (415316,     'Яровое'),
            (415316,  'Ярославль'),
            (415316,     'Ярцево'),
            (415316,  'Ясногорск'),
            (415316,      'Ясный'),
            (415316,     'Яхрома')],
           names=['user_id', 'city_name'], length=409087245)


,user_id,city_name,top_city
0,0,Москва,131
1,1,Люберцы,6
2,1,Москва,572
3,1,Подольск,13
4,1,Санкт-Петербург,91
5,1,Химки,4
6,1,Чехов,14
7,2,Печора,356
8,3,Воронеж,188
9,4,Абинск,1


In [8]:
display(df)
be = BinaryEncoder(cols=['city_name'],return_df=True)
df = be.fit_transform(df)
display(df)

,user_id,city_name,city_count
0,0,Москва,1
1,1,Москва,6
2,2,Печора,1
3,3,Воронеж,1
4,4,Анапа,9
...,...,...,...
415312,415312,Моршанск,1
415313,415313,Саратов,1
415314,415314,Новосибирск,1
415315,415315,Краснодар,1


,user_id,city_name_0,city_name_1,city_name_2,city_name_3,city_name_4,city_name_5,city_name_6,city_name_7,city_name_8,city_name_9,city_count
0,0,0,0,0,0,0,0,0,0,0,1,1
1,1,0,0,0,0,0,0,0,0,0,1,6
2,2,0,0,0,0,0,0,0,0,1,0,1
3,3,0,0,0,0,0,0,0,0,1,1,1
4,4,0,0,0,0,0,0,0,1,0,0,9
...,...,...,...,...,...,...,...,...,...,...,...,...
415312,415312,0,0,1,1,1,0,1,0,0,0,1
415313,415313,0,0,0,0,1,0,0,0,1,1,1
415314,415314,0,0,0,0,0,1,1,0,1,0,1
415315,415315,0,0,0,0,1,0,0,1,0,0,1


In [10]:
df = df.drop('user_id',axis=1)
display(df.info())
print(df['city_count'].max()) #int8

df[['city_count']] = df[['city_count']].astype('int8')
df[df.columns[~df.columns.isin(['city_count'])]] = df[df.columns[~df.columns.isin(['city_count'])]].astype('category')

display(df.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 415317 entries, 0 to 415316
Data columns (total 11 columns):
 #   Column       Non-Null Count   Dtype
---  ------       --------------   -----
 0   city_name_0  415317 non-null  int64
 1   city_name_1  415317 non-null  int64
 2   city_name_2  415317 non-null  int64
 3   city_name_3  415317 non-null  int64
 4   city_name_4  415317 non-null  int64
 5   city_name_5  415317 non-null  int64
 6   city_name_6  415317 non-null  int64
 7   city_name_7  415317 non-null  int64
 8   city_name_8  415317 non-null  int64
 9   city_name_9  415317 non-null  int64
 10  city_count   415317 non-null  int64
dtypes: int64(11)
memory usage: 34.9 MB


None

124
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 415317 entries, 0 to 415316
Data columns (total 11 columns):
 #   Column       Non-Null Count   Dtype   
---  ------       --------------   -----   
 0   city_name_0  415317 non-null  category
 1   city_name_1  415317 non-null  category
 2   city_name_2  415317 non-null  category
 3   city_name_3  415317 non-null  category
 4   city_name_4  415317 non-null  category
 5   city_name_5  415317 non-null  category
 6   city_name_6  415317 non-null  category
 7   city_name_7  415317 non-null  category
 8   city_name_8  415317 non-null  category
 9   city_name_9  415317 non-null  category
 10  city_count   415317 non-null  int8    
dtypes: category(10), int8(1)
memory usage: 4.4 MB


None

In [11]:
df.to_parquet('..\\full_data\\figma_plan\\city.parquet', index=False)
del df

## Производитель binary encoder

In [13]:
path = '..\\full_data\\competition_data_final_pqt'
columns = {'user_id': 'int32', 'cpe_manufacturer_name': 'category'}
data = read_data(path,columns)
display(data.info())
display(data.isna().sum())
display(len(data['cpe_manufacturer_name'].unique()))

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 322899435 entries, 0 to 322899434
Data columns (total 2 columns):
 #   Column                 Dtype   
---  ------                 -----   
 0   user_id                int32   
 1   cpe_manufacturer_name  category
dtypes: category(1), int32(1)
memory usage: 1.5 GB


None

user_id                  0
cpe_manufacturer_name    0
dtype: int64

In [18]:
display(data.isna().sum())
df = data.groupby(['user_id', 'cpe_manufacturer_name'])['cpe_manufacturer_name'].count()
df.name = 'top_brend'

display(df.info())
print(df.index)

df = df[df != 0].reset_index()
display(df.head(20))

df = df.sort_values(by='top_brend', ascending=False).drop_duplicates(subset='user_id', keep='first', ignore_index=True)
df = df.drop('top_brend', axis=1)
df = df.sort_values(by='user_id').reset_index(drop=True)

user_id                  0
cpe_manufacturer_name    0
dtype: int64

<class 'pandas.core.series.Series'>
MultiIndex: 15366729 entries, (0, 'Alcatel') to (415316, 'ZTE')
Series name: top_brend
Non-Null Count     Dtype
--------------     -----
15366729 non-null  int64
dtypes: int64(1)
memory usage: 209.8 MB


None

MultiIndex([(     0,                           'Alcatel'),
            (     0,                             'Apple'),
            (     0,                              'Asus'),
            (     0,                         'Atlas LLC'),
            (     0,                'BQ Devices Limited'),
            (     0,                         'Blackview'),
            (     0,   'Doke Communication (HK) Limited'),
            (     0,                            'Doogee'),
            (     0,                        'Google Inc'),
            (     0,                               'HTC'),
            ...
            (415316,                           'Samsung'),
            (415316,                              'Sony'),
            (415316,   'Sony Mobile Communications Inc.'),
            (415316,                             'Tecno'),
            (415316, 'Umi Network Technology Co Limited'),
            (415316,      'Vingroup Joint Stock Company'),
            (415316,                    

,user_id,cpe_manufacturer_name,top_brend
0,0,Samsung,131
1,1,Xiaomi,700
2,2,Huawei,356
3,3,Huawei Device Company Limited,188
4,4,Huawei,591
5,5,Apple,2519
6,6,Huawei,456
7,7,Huawei,461
8,8,Apple,976
9,9,Apple,315


In [22]:
display(df)
be = BinaryEncoder(cols=['cpe_manufacturer_name'],return_df=True)
df = be.fit_transform(df)
display(df)

,user_id,cpe_manufacturer_name
0,0,Samsung
1,1,Xiaomi
2,2,Huawei
3,3,Huawei Device Company Limited
4,4,Huawei
...,...,...
415312,415312,Huawei
415313,415313,Xiaomi
415314,415314,Huawei Device Company Limited
415315,415315,Huawei


,user_id,cpe_manufacturer_name_0,cpe_manufacturer_name_1,cpe_manufacturer_name_2,cpe_manufacturer_name_3,cpe_manufacturer_name_4,cpe_manufacturer_name_5
0,0,0,0,0,0,0,1
1,1,0,0,0,0,1,0
2,2,0,0,0,0,1,1
3,3,0,0,0,1,0,0
4,4,0,0,0,0,1,1
...,...,...,...,...,...,...,...
415312,415312,0,0,0,0,1,1
415313,415313,0,0,0,0,1,0
415314,415314,0,0,0,1,0,0
415315,415315,0,0,0,0,1,1


In [23]:
df = df.drop('user_id',axis=1)
display(df.info())

df[df.columns] = df[df.columns].astype('category')

display(df.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 415317 entries, 0 to 415316
Data columns (total 6 columns):
 #   Column                   Non-Null Count   Dtype
---  ------                   --------------   -----
 0   cpe_manufacturer_name_0  415317 non-null  int64
 1   cpe_manufacturer_name_1  415317 non-null  int64
 2   cpe_manufacturer_name_2  415317 non-null  int64
 3   cpe_manufacturer_name_3  415317 non-null  int64
 4   cpe_manufacturer_name_4  415317 non-null  int64
 5   cpe_manufacturer_name_5  415317 non-null  int64
dtypes: int64(6)
memory usage: 19.0 MB


None

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 415317 entries, 0 to 415316
Data columns (total 6 columns):
 #   Column                   Non-Null Count   Dtype   
---  ------                   --------------   -----   
 0   cpe_manufacturer_name_0  415317 non-null  category
 1   cpe_manufacturer_name_1  415317 non-null  category
 2   cpe_manufacturer_name_2  415317 non-null  category
 3   cpe_manufacturer_name_3  415317 non-null  category
 4   cpe_manufacturer_name_4  415317 non-null  category
 5   cpe_manufacturer_name_5  415317 non-null  category
dtypes: category(6)
memory usage: 2.4 MB


None

In [24]:
df.to_parquet('..\\full_data\\figma_plan\\cpe_manufacturer.parquet', index=False)
del df

## Количество моделей

In [3]:
path = '..\\full_data\\competition_data_final_pqt'
columns = {'user_id': 'int32', 'cpe_model_name': 'category'}
data = read_data(path,columns)
display(data.info())
display(data.isna().sum())
display(len(data['cpe_model_name'].unique()))

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 322899435 entries, 0 to 322899434
Data columns (total 2 columns):
 #   Column          Dtype   
---  ------          -----   
 0   user_id         int32   
 1   cpe_model_name  category
dtypes: category(1), int32(1)
memory usage: 1.8 GB


None

user_id           0
cpe_model_name    0
dtype: int64

599

In [12]:
df = data.groupby(['user_id', 'cpe_model_name'])['cpe_model_name'].count()
df.name = 'top_model'

display(df.info())
print(df.index)

df = df[df != 0].reset_index()
display(df.head(20))

model_count = df.groupby('user_id', group_keys=False)['cpe_model_name'].count()
df['model_count'] = model_count

<class 'pandas.core.series.Series'>
MultiIndex: 248774883 entries, (0, '1') to (415316, 'iPhone Xs Max')
Series name: top_model
Non-Null Count      Dtype
--------------      -----
248774883 non-null  int64
dtypes: int64(1)
memory usage: 3.3 GB


None

MultiIndex([(     0,              '1'),
            (     0,         '1 Dual'),
            (     0,  '11 Lite 5G NE'),
            (     0,            '11T'),
            (     0,        '15 Lite'),
            (     0,             '16'),
            (     0,           '16th'),
            (     0,           '1904'),
            (     0,             '1S'),
            (     0,             '1X'),
            ...
            (415316,       'iPhone 7'),
            (415316,  'iPhone 7 Plus'),
            (415316,       'iPhone 8'),
            (415316,  'iPhone 8 Plus'),
            (415316,      'iPhone SE'),
            (415316, 'iPhone SE 2020'),
            (415316,       'iPhone X'),
            (415316,      'iPhone XR'),
            (415316,      'iPhone Xs'),
            (415316,  'iPhone Xs Max')],
           names=['user_id', 'cpe_model_name'], length=248774883)


,user_id,cpe_model_name,top_model
0,0,Galaxy J1 2016 LTE Dual,131
1,1,Mi 9,700
2,2,Honor 9 Lite,356
3,3,P Smart 2021,188
4,4,Nova 3,591
5,5,iPhone 8 Plus,2519
6,6,Honor 8A,456
7,7,P20 Lite,461
8,8,iPhone XR,976
9,9,iPhone XR,315


In [14]:
df['model_count'].value_counts()

1    415317
Name: model_count, dtype: int64

#### У всех пользователей оказалось по одной модели, поэтому столбец неинформативен

## Тип устройства binary encoder

In [15]:
path = '..\\full_data\\competition_data_final_pqt'
columns = {'user_id': 'int32', 'cpe_type_cd': 'category'}
data = read_data(path,columns)
display(data.info())
display(data.isna().sum())
display(len(data['cpe_type_cd'].unique()))

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 322899435 entries, 0 to 322899434
Data columns (total 2 columns):
 #   Column       Dtype   
---  ------       -----   
 0   user_id      int32   
 1   cpe_type_cd  category
dtypes: category(1), int32(1)
memory usage: 1.5 GB


None

user_id        0
cpe_type_cd    0
dtype: int64

4

In [19]:
df = data.drop_duplicates(subset='user_id',ignore_index=True)

display(df)
be = BinaryEncoder(cols=['cpe_type_cd'],return_df=True)
df = be.fit_transform(df)
display(df)

,user_id,cpe_type_cd
0,45098,smartphone
1,117132,smartphone
2,79395,smartphone
3,91294,smartphone
4,161323,smartphone
...,...,...
415312,375972,smartphone
415313,160996,smartphone
415314,5505,smartphone
415315,260127,smartphone


,user_id,cpe_type_cd_0,cpe_type_cd_1,cpe_type_cd_2
0,45098,0,0,1
1,117132,0,0,1
2,79395,0,0,1
3,91294,0,0,1
4,161323,0,0,1
...,...,...,...,...
415312,375972,0,0,1
415313,160996,0,0,1
415314,5505,0,0,1
415315,260127,0,0,1


In [20]:
df = df.drop('user_id',axis=1)
display(df.info())

df[df.columns] = df[df.columns].astype('category')

display(df.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 415317 entries, 0 to 415316
Data columns (total 3 columns):
 #   Column         Non-Null Count   Dtype
---  ------         --------------   -----
 0   cpe_type_cd_0  415317 non-null  int64
 1   cpe_type_cd_1  415317 non-null  int64
 2   cpe_type_cd_2  415317 non-null  int64
dtypes: int64(3)
memory usage: 9.5 MB


None

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 415317 entries, 0 to 415316
Data columns (total 3 columns):
 #   Column         Non-Null Count   Dtype   
---  ------         --------------   -----   
 0   cpe_type_cd_0  415317 non-null  category
 1   cpe_type_cd_1  415317 non-null  category
 2   cpe_type_cd_2  415317 non-null  category
dtypes: category(3)
memory usage: 1.2 MB


None

In [21]:
df.to_parquet('..\\full_data\\figma_plan\\cpe_type.parquet', index=False)
del df

## Цена устройства

In [24]:
path = '..\\full_data\\competition_data_final_pqt'
columns = {'user_id': 'int32', 'price': 'float32'}
data = read_data(path,columns)
display(data.info())
display(data.isna().sum())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 322899435 entries, 0 to 322899434
Data columns (total 2 columns):
 #   Column   Dtype  
---  ------   -----  
 0   user_id  int32  
 1   price    float32
dtypes: float32(1), int32(1)
memory usage: 2.4 GB


None

user_id          0
price      6589482
dtype: int64

In [26]:
df = data.drop_duplicates(subset='user_id',ignore_index=True)
display(df)
display(df.isna().sum())

,user_id,price
0,45098,20368.0
1,117132,4990.0
2,79395,74259.0
3,91294,23876.0
4,161323,20465.0
...,...,...
415312,375972,56357.0
415313,160996,26154.0
415314,5505,15490.0
415315,260127,40612.0


user_id        0
price      10754
dtype: int64